In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
assert 'GPU' in str(device_lib.list_local_devices())
from tensorflow import GPUOptions, Session, ConfigProto

# confirm Keras sees the GPU
from keras import backend
assert len(backend.tensorflow_backend._get_available_gpus()) > 0

gpu_options = GPUOptions(per_process_gpu_memory_fraction = 0.90)           #Assign portion of memory for training this model
sess = Session(config = ConfigProto(gpu_options = gpu_options))


#Uncomment to run on CPU
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

c:\users\manap\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\manap\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\manap\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\manap\appd

Using TensorFlow backend.


In [2]:
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from datetime import datetime
now=datetime.now()
s1 = now.strftime("%m\%d\%Y\%H\%M\%S")
print(s1)
NAME = "Galaxy-zoo-"+s1


DIR = "images_training_rev1"

train_path = DIR + "\Train"
validate_path = DIR + "\Validate"

df = pd.read_csv('training_solutions_rev1.csv')

df.set_index('GalaxyID', inplace = True)

09\10\2019\22\15\08


In [3]:

def img_processor(path):
    img = cv2.imread(path)
    img = img[106:106*3,106:106*3,:]
    img = cv2.resize(img,(106,106), interpolation = cv2.INTER_CUBIC)
    return img

def Batch_generator(DIR):
    for img in os.listdir(DIR):
        im = img_processor(os.path.join(DIR,img))
        ind = int(img[:-4])
        y_train = df.loc[ind].values
        X_train = np.array(im)
        X_train = X_train.reshape(1,106,106,3)
        y_train = y_train.reshape(1,37)
        yield(X_train,y_train)
        
def Validation_generator(DIR):
    for img in os.listdir(DIR):
        im = img_processor(os.path.join(DIR,img))
        ind = int(img[:-4])
        y_valid = df.loc[ind].values
        X_valid = np.array(im)
        X_valid = X_valid.reshape(1,106,106,3)
        y_valid = y_valid.reshape(1,37)
        yield(X_valid,y_valid)

In [4]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,ZeroPadding2D,Lambda
from keras.optimizers import SGD, RMSprop, Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard

In [5]:
def Conv(model,layers,filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1,1))) 
        model.add(Conv2D((filters), kernel_size = (3,3), activation = 'relu'))
        
    model.add(MaxPooling2D(pool_size=(2,2), strides = (2,2)))
    
def FC_block(model,filters):
    model.add(Dense(filters, activation='relu'))
    model.add(Dropout(0.5))
    
def VGG16():
    model = Sequential()
    model.add(Lambda(lambda x : x, input_shape=(106,106,3)))

    Conv(model,2,64)
    Conv(model,2,128)
    Conv(model,3,256)
    Conv(model,3,512)
    Conv(model,3,512)
    
    model.add(Flatten())
    FC_block(model,4096)
    FC_block(model,4096)
    FC_block(model,1000)
    
    model.add(Dense(37))
    model.add(Activation("sigmoid"))
    if os.path.exists('temp.hdf5'):
        model.load_weights("temp.hdf5")
   
    return model

In [6]:
model = VGG16()

model.compile(optimizer = RMSprop(lr=1e-6), loss='mean_squared_error', metrics = ['accuracy'])

#loss, acc = new_model.evaluate(test_images, test_labels)
#print("Restored model, accuracy: {:5.2f}%".format(100*acc))


#loss, acc = model.evaluate(Batch_generator(train_path))
#print("Restored model, accuracy: {:5.2f}%".format(100*acc))
        



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [7]:
batch_size = 64

steps_per_batch = int(len(os.listdir(train_path))/batch_size)
val_steps_per_batch = int(len(os.listdir(validate_path))/batch_size) 

In [ ]:
checkpointer = ModelCheckpoint(filepath='temp.hdf5', verbose=1, save_best_only=True)

tensorboard = TensorBoard('logs\{}'.format(NAME))
#tensorboard --logdir=data/ --host localhost --port 8088   
#CMD instruction, local host is http://localhost:8088/

model.fit_generator(Batch_generator(train_path),
                    samples_per_epoch=steps_per_batch,
                    epochs = 50,
                    validation_data=Validation_generator(validate_path),
                    nb_val_samples=val_steps_per_batch,
                    verbose = 1,callbacks=[checkpointer,tensorboard]
                   )




Epoch 1/50
715/962 [=====================>........] - ETA: 14s - loss: 0.0316 - acc: 0.5678